# Interpretation

In [15]:
class Evolution:
    def __init__(self, k, c1, c2):
        self.k = k
        self.c1 = c1
        self.c2 = c2

## Trend Delay

In [1]:
import os
import json
from entities import TimeWindow

dataset = 'youtube'
layer = 'TrendDelayLayer'


time_windows = []

for r, _, files in os.walk(f'data/{dataset}/raw/timeslices/{layer}'):
    # sort by year, week
    files.sort(key=lambda name: [int(n) for n in name[:-5].split('_')])

    for f in files:
        with open(os.path.join(r, f), 'r') as fh:
            clusters = json.loads(fh.read())

            time_window = TimeWindow.create_from_serializable_dict(clusters)
            time_windows.append(time_window)

In [11]:
feature_name = 'trend_delay'

from typing import List
from processing import ClusterMetricsCalculator1D

class Cluster:
    def __init__(self, nodes: List):
        
        calc = ClusterMetricsCalculator1D(nodes, feature_name, 0 , 0)

        self.nodes = nodes
        self.center = calc.get_center()
        self.size = calc.get_size()


In [7]:
ts_old = 1

In [12]:
time_window = time_windows[ts_old]

clusters = {}

for k, nodes in time_window.clusters.items():
    clusters[k] = Cluster(nodes)


In [13]:
time_window.time

'(2017, 47)'

In [14]:
time_window = time_windows[ts_old+1]
    
clusters2 = {}

for k, nodes in time_window.clusters.items():
    clusters2[k] = Cluster(nodes)


In [27]:
threshold = 500
large_keys = [k for k, c in clusters.items() if c.size >= threshold] + [k for k, c in clusters2.items() if c.size >= threshold]
len(large_keys)

9

In [35]:
from typing import Dict

large_keys = [k for k, c in clusters.items() if c.size >= threshold] + [k for k, c in clusters2.items() if c.size >= threshold]
large_keys = list(set(large_keys))
large_keys.sort()

old:Dict[str, Cluster] = {}
new:Dict[str, Cluster] = {}

for k in large_keys:
    c1 = clusters[k]
    c2 = clusters2[k]

    old[k] = c1
    new[k] = c2

In [36]:
len(old)

5

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots()

cols = ['#4d4d4d', '#8cdcda', '#b1d877', '#f16a70', 'orange']
size_factor = 1

# old 
sc = ax.scatter(
    [c.center for c in old],# [c.center[0] for c in c_combined],
    [0 for c in c_combined],
    c=cols,
    s=[c.size*size_factor for c in c_combined], 
    alpha = 1)

# new 
sc = ax.scatter(
    range(len(c2_combined)), #[c.center[0] for c in c2_combined],
    [1 for c in c2_combined],
    c=cols,
    s=[c.size*size_factor for c in c2_combined], 
    alpha = 1)

plt.yticks([0, 1], ['t', 't+1'])
plt.ylim([-.5, 2.5])

plt.xticks([0, 1, 2, 3, 4], coords, rotation=45)
plt.xlim([-1, 5])

plt.ylabel("Time window")
plt.xlabel("Start location (latitude, longitude)")

labels = ['Train station Campanha', 'Hospital São João', 'Bus station Internorte', 'Port Marina da Afurada', 'Hospital Santo António']

legend_patches = []
for c, l in zip(cols, labels):
    pat = mpatches.Patch(color=c, label=l)
    legend_patches.append(pat)

plt.legend(handles=legend_patches)
plt.show()
plt.show()

